## Prepare Project

Install the necessary requirements (can be run on Google Colab).

In [ ]:
!pip install -r /content/requirements.txt

Import libraries

In [ ]:
from time import time
from collections import Counter

import graphviz
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz

from joblib import dump, load
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.feature_selection import SelectFromModel, VarianceThreshold, RFECV, SequentialFeatureSelector
from sklearn.model_selection import GridSearchCV, KFold, RepeatedKFold, cross_validate, train_test_split, validation_curve
from sklearn.metrics import multilabel_confusion_matrix, classification_report, roc_curve, auc, accuracy_score, hamming_loss, make_scorer

import pingouin as pg
import scikit_posthocs as sp
from mlxtend.evaluate import mcnemar
from mlxtend.evaluate import mcnemar_table

sns.set()
import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning)

## Define helper functions and pipelines

### Feature selection functions

In [ ]:
def qconstant_filtering(X, thresh=0.01):
  '''Remove constant and quasi-constant features.'''

  qconstant_filter = VarianceThreshold()
  qconstant_filter.fit(X)

  qfeatures = X.iloc[:, qconstant_filter.variances_ <= thresh].columns

  print(f"Quasi-Constant features found: {len(qfeatures)}")

  X = X.drop(labels=qfeatures, axis=1)

  print(f"New feature shape: {X.shape}")
  return X


def correlation_filtering(X, thresh=0.9, method='spearman'):
    '''Remove correlated features.'''

    # Set of all the names of deleted columns
    col_corr = set()

    # Create correlation matrix from the dataset
    corr_matrix = X.corr(method=method).abs()

    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= thresh) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in X.columns:
                    del X[colname] # deleting the column from the dataset

    print(f"Correlated features found: {len(col_corr)}")
    print(f"New feature shape: {X.shape}")
    return X

def feature_importance(model, X, Y, thresh='1.5*mean'):
    '''Select features based on importance weights.'''

    # Initialize and apply transformer
    forest_sel = SelectFromModel(model, threshold=thresh)

    tic = time()

    forest_sel.fit(X, Y)

    toc = time()

    # Transform X feature set
    X = X.iloc[:, forest_sel.get_support()]

    print(f"Done in {(toc - tic)/60:0.2f} minutes")
    print(f"New feature shape: {X.shape}")
    return X

def rfecv_selection(model, X, Y, cv=10, scoring=None, figure=False):
    '''Feature ranking with recursive feature elimination and
    cross-validated selection of the best number of features.'''

    # Initialize and apply RFECV
    rfecv = RFECV(estimator=model, step=1, cv=cv, scoring=scoring, n_jobs=-1)

    tic = time()

    rfecv.fit(X, Y)

    toc = time()

    # Transform X feature set
    X = X.iloc[:, rfecv.support_]

    print(f"Done in {(toc - tic)/60:0.2f} minutes")
    print(f"Optimal number of features: {rfecv.n_features_}")
    print(f"New feature shape: {X.shape}")

    # Plot RFECV search
    if figure == True:

        # Plot number of features VS. cross-validation scores
        plt.figure(figsize=(16,9))


        plt.xticks(np.arange(0, 56, 5).astype(int))
        plt.xlabel("Number of features selected", fontsize=15)
        plt.ylabel("Cross validation score (Accuracy)", fontsize=15)
        plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)

        plt.savefig('RFECV.png')
        plt.show()

    return X

def forward_selection(model, X, Y, cv=10, n_feat=10, scoring=None, feat_only=False):
    '''Perform Sequential Feature Selection in the forward direction.'''

    # Initialize and apply Sequential Forward selection
    sfs_forward = SequentialFeatureSelector(model, n_features_to_select=n_feat, cv=cv,
                                            direction='forward', scoring=scoring, n_jobs=-1)

    tic = time()

    sfs_forward.fit(X, Y)

    toc = time()

    # Transform X and get feature set
    X = X.iloc[:, sfs_forward.get_support()]
    features = X.columns.to_list()

    print(f"Done in {(toc - tic)/60:0.2f} minutes")

    if feat_only == True:
        return features

    print(f"New feature shape: {X.shape}")
    print(f"Features selected by sequential forward selection: {features}")
    return X

def manual_selection(features, thresh=0.5):
    '''Manually select robust features by calculating overlap.'''

    # Calculate feature overlap
    count_overlap = Counter(x for xs in features for x in set(xs))

    # Initialize list to collect features
    sel_feat = []
    start = len(features)
    stop = int(len(features)*thresh)


    for i in range(start, stop, -1):
        for k, v in count_overlap.items():
            if v == i and len(sel_feat) < 10:
                sel_feat.append(k)

    print(f'Selected features with good overlap: {sel_feat}')

    return sel_feat

### Evaluation functions

In [ ]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''Compute the Hamming score (i.e., label-based accuracy) for the multi-label case.'''

    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

def model_comparison(models, X, Y, cv=10, scoring=None):
    '''Model comparison using dictionary with different pipelines.'''

    # Initialize lists to store results
    names = []
    results = []

    # k-fold CV results
    for key, value in models.items():
        cv_results = cross_validate(value['pl'], X, Y, cv=cv, scoring = scoring, n_jobs=4)
        results.append(cv_results)
        names.append(value['name'])

        if scoring:
            print(f"\n{value['name']:<11}: Exact Match Ratio =    {cv_results['test_Exact Match Ratio'].mean():0.3f} ({cv_results['test_Exact Match Ratio'].std():0.3f})")
            print(f"{' ':<11}  Label-based Accuracy = {cv_results['test_Label-based Accuracy'].mean():0.3f} ({cv_results['test_Label-based Accuracy'].std():0.3f})")
        else:
            print(f"{value['name']:<11}: {cv_results['test_score'].mean():0.3f} ({cv_results['test_score'].std():0.3f})")

    return results, names

def feature_comparison(model, X, Y, cv=10, scoring=None, X_names=None, save=True):
    '''Perform cross-validation using the same classifier with different feature sets.'''

    # Initialize dataframe to store results
    ds_result = pd.DataFrame()

    # Perform cross-validation and save the results
    if X_names:
        for name, data in zip(X_names, X):
            cv_results = cross_validate(model, data, Y, cv=cv, scoring=scoring, n_jobs=4)
            ds_result[name] = cv_results['test_score']
    else:
        for idx, data in enumerate(X):
            cv_results = cross_validate(model, data, Y, cv=cv, scoring=scoring, n_jobs=4)
            ds_result[idx] = cv_results['test_score']

    if save == True:
        ds_result.to_csv('Performance of different feature sets in CV.csv', index=None)

    return ds_result

def accuracy_per_class(y_true, y_pred):
    '''Calculate accuracy per class.'''

    # Compute confusion matrix for the multilabel case
    cm = multilabel_confusion_matrix(y_true, y_pred)

    # Calculate accuracy for each label (TP + TN)/(TP + FP + TN + FN)
    gc_acc = cm[0].diagonal().sum()/cm[0].sum()
    lc_acc = cm[1].diagonal().sum()/cm[1].sum()

    return (gc_acc, lc_acc)

def general_classification_report(y_true, y_pred, title='Classification Report'):
    '''Show a Classification Report with accuracy.'''

    print(title)
    print('-'*len(title))
    print(classification_report(y_true, y_pred, zero_division=0))

    print("\nAccuracy per class")
    print(f'GC = {accuracy_per_class(y_true, y_pred)[0]:0.3f}')
    print(f'LC = {accuracy_per_class(y_true, y_pred)[1]:0.3f}')

    print(f'\nLabel-based accuracy = {hamming_score(y_true, y_pred):0.3f}')

    return classification_report(y_true, y_pred, zero_division=0, output_dict=True)

def show_confusion_matrix(confusion_matrix, axes, class_label, class_names, fontsize=14):
    '''Calculate and plot confusion matrix.'''

    # Plot heatmap from confusion matrix
    ds_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names)

    try:
        heatmap = sns.heatmap(ds_cm, annot=True, fmt="d", cbar=False, ax=axes, cmap='Blues')
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")

    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    axes.set_ylabel('True label')
    axes.set_xlabel('Predicted label')
    axes.set_title("Confusion Matrix for the " + class_label + " class", fontsize=14)

def plot_roc_curve(y_true, y_pred1, y_pred2, model_names, class_names=None, title='ROC curve', save=False):
    '''Calculate and plot ROC curve.'''

    # Get number of classes/labels
    n_classes = y_true.shape[1]

    # Compute ROC curve and ROC area for each class
    fpr, tpr, roc_auc = dict(), dict(), dict()

    for i in range(0, n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_pred1[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    for i in range(n_classes, 2*n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true[:, i-n_classes], y_pred2[:, i-n_classes])
        roc_auc[i] = auc(fpr[i], tpr[i])

    plt.figure(figsize=(12, 8))

    if class_names:
        for i in range(0, 2*n_classes):
          plt.plot(fpr[i], tpr[i], lw=2,
                    label=f'{model_names[i]}: {class_names[i%2]} class (AUC = {roc_auc[i]:0.3f})')
    else:
        for i in range(2*n_classes):
            plt.plot(fpr[i], tpr[i], lw=2,
                     label=f'{model_names[i]}: Class {i%2} (AUC = {roc_auc[i]:0.3f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc='best')

    if save:
        plt.savefig(title + '.png')

    plt.show()

### Optimization functions

In [ ]:
def tree_optimization(model, param_grid, X, Y, cv=10, scoring=None):
    '''Optimize Decision Tree Classifier using Grid Search.'''

    # Initialize and apply GridSearchCV
    grid_cv = GridSearchCV(model, param_grid, cv=cv, verbose=1, n_jobs=-1, scoring=scoring)

    tic = time()

    grid_cv.fit(X, Y)

    toc = time()

    print(f"Done in {(toc - tic)/60:0.2f} minutes")
    print(f"Best estimator: {grid_cv.best_estimator_} with score {grid_cv.best_score_}")

    return grid_cv.best_estimator_

def plot_validation_curve(estimator, X, y, param_name, param_range = np.arange(1, 30), ylim=(0.0, 1.1), cv=10, scoring='accuracy', save=False):
    '''Generate a plot of the validation curve for a given hyperparameter.'''

    # Calculate scores
    train_scores, test_scores = validation_curve(
                                estimator, X, y, param_name=param_name, cv=cv, 
                                param_range=param_range,
                                scoring=scoring)

    train_scores_mean = np.mean(train_scores, axis = 1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis = 1)
    test_scores_std = np.std(test_scores, axis=1)

    # Plotting options
    plt.figure(figsize=(18, 9))
    plt.xlabel(param_name, fontsize=15)
    plt.xticks(param_range, fontsize=13)
    plt.yticks(np.arange(0, 1.1, 0.1), fontsize=13)
    plt.ylabel("Hamming Score", fontsize=15)
    plt.ylim(*ylim)

    plt.plot(param_range, train_scores_mean, label="Training score", color="r")
    plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=2)
    plt.plot(param_range, test_scores_mean, label="Test score", color="g")
    plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=2)
    plt.legend(loc="best", fontsize=15)

    if save == True:
        plt.savefig('Decision Tree Validation Curve (max_depth).png')

    return plt

### Statistical functions

In [ ]:
def friedman_test(dataset, alpha=0.05, long=False, effect_size=True):
    '''Perform the Friedman test on a provided dataset.'''

    # Transform dataframe to the appropriate format
    if not long:
        dataset = dataset.melt(ignore_index=False).reset_index(drop=False)
        dataset = dataset.rename(columns={'index': 'Fold', 'variable': 'Feature set', 'value': 'Accuracy'})

    # Perform the Friedman test
    result = pg.friedman(dataset, dv='Accuracy', within='Fold', subject='Feature set')

    fried, p_value = result['Q'][0], result['p-unc'][0]

    print(f'Friedman statistic: {fried:0.2f}')
    print(f'p-value: {p_value:0.6f}')

    if p_value < alpha:
        print('\nThe differences between some of the groups are statistically significant.')
        print('Use a post hoc test to determine which classifiers actually differ.')
    else:
        print('\nThe differences between the groups are not statistically significant.')

    # Calculate effect size
    if effect_size == True:
        print('\nRelevant effect size')
        print('----------------------')
        print(f"Kendall's W coefficient: {result['W'][0]:0.3f}")

    return result

def nemeyi_test(dataset, alpha=0.05, significant_only=False):
    '''Perform the Nemeyi Post Hoc test on a provided dataset.'''

    # Perform the Nemenyi test
    ds_nemeyi = sp.posthoc_nemenyi_friedman(dataset)

    if significant_only == True:
        print(f'Returning only statistically significant differences (p value < {alpha})')

        return ds_nemeyi[ds_nemeyi < alpha]

    return ds_nemeyi

def wilcoxon_test(x, y, alpha=0.05, alternative='two-sided', effect_size=True):
    '''Perform the Wilcoxon-Signed Rank test on two sets of observations.'''

    # Perform the Wilcoxon Test
    result = pg.wilcoxon(x, y, tail=alternative)

    wilcoxon, p_value = result['W-val'][0], result['p-val'][0]

    print(f'Wilcoxon statistic: {wilcoxon:0.2f}')
    print(f'p-value: {p_value:0.6f}')

    if p_value < alpha:
        print('\nThe differences between measurements are statistically significant.')
    else:
        print('\nThe differences between measurements are not statistically significant.')

    # Calculate effect size
    if effect_size == True:
        print('\nRelevant effect sizes')
        print('-----------------------')
        print(f"Rank-biserial correlation: {result['RBC'][0]:0.3f}")
        print(f"Common language effect size: {result['CLES'][0]:0.3f}")

    return result

def mcnemar_test(y_true, y_pred1, y_pred2, visualize=True, save=True, title="McNemar's table", model_names=None, effect_size=True):
    '''Calculate McNemar's test to compare two classifiers.'''

    # Compute 2x2 contingency table
    table = mcnemar_table(y_true, y_pred1, y_pred2)

    # Plot 2x2 contingency table
    if visualize == True:
        if model_names:
            col_labels=['Rule-based Classifier Correct', 'Rule-based Classifier Incorrect']
            row_labels=['Decision Tree Classifier Correct', 'Decision Tree Classifier Incorrect']
        else:
            col_labels=['Model 1 Correct', 'Model 1 Incorrect']
            row_labels=['Model 2 Correct', 'Model 2 Incorrect']

        plt.figure(figsize=(12, 9))

        ds_table = pd.DataFrame(table, index=row_labels, columns=col_labels)

        heatmap = sns.heatmap(ds_table, annot=True, fmt="d", cbar=False, cmap='Blues')

        plt.yticks([0.2, 1.15])

        plt.title(title)

        if save == True:
            plt.savefig(title + '.png')

    # Calculate test statistic and corresponding p-value
    chi2, p = mcnemar(table)

    print(f'Chi-squared statistic: {chi2:0.2f}')
    print(f'p-value: {p:0.6f}')

    # Calculate effect size
    if effect_size == True:

        # Calculate odds ratio (OR)
        b = table[0, 1]
        c = table[1, 0]
        OR = max(b/c, c/b)

        print('\nRelevant effect size')
        print('----------------------')
        print(f'Odds Ratio: {OR:0.2f}')

        return pd.DataFrame({'chi2': chi2, 'p-val': p, 'OR': OR}, index=['McNemar'])

    return pd.DataFrame({'chi2': chi2, 'p-val': p}, index=['McNemar'])

### Baseline classifiers

In [ ]:
class RulePredict:


    def __init__(self, multilabel=True, GC=None, LC=None):
        '''Initialize Rule-based Predictor.'''

        self.GC = GC
        self.LC = LC
        self.multilabel = multilabel

    def predict(self, X):
        '''Predict GC-LC class for the input X.'''

        if self.multilabel == True:
            return np.array([self.gc_predict(X), self.lc_predict(X)])

        elif self.GC == True:
            return self.gc_predict(X)

        elif self.LC == True:
            return self.lc_predict(X)

        else:
            raise ValueError('Invalid input. Choose a valid method (multilabel, GC, LC).')

    def gc_predict_proba(self, X):
        '''Predict GC class probabilities of the input samples X.'''

        prob = 0

        if 100 <= X['BoilingPoint'] <= 350:
            prob += 1/3
        if 2 <= X['logP']:
            prob += 1/3
        if X['MW'] <= 700:
            prob += 1/3

        return prob

    def gc_predict(self, X, threshold=0.5):
        '''Predict GC class value for X.'''


        prob = self.gc_predict_proba(X)

        return 1 if prob >= threshold else 0

    def lc_predict_proba(self, X):
        '''Predict LC class probabilities of the input samples X.'''

        prob = 0

        if X['logP'] <= 5.91:
            prob += 1

        return prob

    def lc_predict(self, X, threshold=0.5):
        '''Predict LC class value for X.'''

        prob = self.lc_predict_proba(X)

        return 1 if prob >= threshold else 0

In [ ]:
model_dict = \
    {
    # experiments to build pipeline
    # Note: keys are of the form model_*, which are used to execute the associated values of 'pl' keys
    'model_1': {
        'name': 'Dummy',
        'pl': Pipeline([ ('dummy_clf', DummyClassifier(strategy='stratified', random_state=42)) ])
        },
    # systematic check of default classifiers + scaling
    'model_2': {
        'name': 'KNN',
        'pl': Pipeline([ ('knn', KNeighborsClassifier(n_jobs=-1)) ])
        },
    'model_3': {
        'name': 'Scaled KNN',
        'pl': Pipeline([ ('scaling', StandardScaler()),
                        ('knn', KNeighborsClassifier(n_jobs=-1)) ])
        },
    'model_4': {
        'name': 'DT',
        'pl': Pipeline([ ('decision-tree', DecisionTreeClassifier(random_state=42)) ])
        },
    'model_5': {
        'name': 'Scaled DT',
        'pl': Pipeline([ ('scaling', StandardScaler()),
                        ('decision-tree', DecisionTreeClassifier(random_state=42)) ])
        },
    'model_6': {
        'name': 'RF',
        'pl': Pipeline([ ('random-forest', RandomForestClassifier(random_state=42, n_estimators=100)) ])
        },
    'model_7': {
        'name': 'Scaled RF',
        'pl': Pipeline([ ('scaling', StandardScaler()),
                        ('random-forest', RandomForestClassifier(random_state=42, n_estimators=100)) ])
        }
    }

## Load and preprocess data

In [ ]:
path = 'http://users.uoa.gr/~nalygizakis/Applied%20Data%20Science/GC_LC%20dataset.xlsx'
ds = pd.read_excel(path)

In [ ]:
ds.head()

Check for null values

In [ ]:
print(f'Null values per feature: \n\n{ds.iloc[:, 7:].isna().sum()}')
print(f'\nTotal null values: {ds.iloc[:, 7:].isna().sum().sum()}')

Check for duplicated entries

In [ ]:
print(f'Total duplicated entries: {int(ds[ds.duplicated()].sum().sum())}')

Define feature set and target variable for multilabel problem

In [ ]:
# Initial feature set
X = ds.iloc[:, 9:]

# Target variable (multilabel)
mlb = MultiLabelBinarizer()
mlb.fit([['GC', 'LC']])
y = mlb.transform(ds['Response'].str.split(';'))

print(X.shape, y.shape)

(6431, 1446) (6431, 2)


Check data types

In [ ]:
X.dtypes.value_counts()

To simulate a realistic application we will split into Train and Test set and ignore completely the Test set from the start. We will only use it after completing the creation process to evaluate our model's generalization. 

This way, the Test set will be an unbiased estimator of the true error (the test error will approximate the true generalization error).

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state=42, stratify=y)

print(f'X_train = {X_train.shape}, Y_train = {Y_train.shape}, X_test = {X_test.shape}, Y_test = {Y_test.shape}')

X_train = (5144, 1446), Y_train = (5144, 2), X_test = (1287, 1446), Y_test = (1287, 2)


## Feature selection

Apply various feature selection methods to find an accurate and interpretable feature set.

### Filter methods

Remove constant/quasi-constant features

In [ ]:
X_vt = qconstant_filtering(X_train)

Correlation feature selection

In [ ]:
X_corr = X_vt.copy()
X_corr = correlation_filtering(X_corr) 

### Random Forest feature importances

Select features based on random forest importance

In [ ]:
model = RandomForestClassifier(n_estimators = 100, random_state=42)

X_forest = feature_importance(model, X_corr, Y_train)

### RFECV

Find the optimal number of features using RFECV.

In [ ]:
# Model and CV split
k_fold = KFold(n_splits=10, shuffle = True, random_state = 200)
model = RandomForestClassifier(n_estimators = 100, random_state=200)

# Scoring (label-based accuracy)
lab_accu = make_scorer(hamming_score)

X_rfe = rfecv_selection(model, X_forest, Y_train, cv=k_fold, scoring=lab_accu, figure=True)

### Sequential feature selection

Calculate optimal feature subsets using sequential forward selection.

**Warning**: the following cell may take a very long time (possibly 6-7 hours depending on your hardware).

In [ ]:
# Define list for results
feat_10 = []

# Scoring (label-based accuracy)
lab_accu = make_scorer(hamming_score)

for i in range(0, 5):bet
    k_fold = KFold(n_splits=10, shuffle = True, random_state = i)
    model = RandomForestClassifier(n_estimators = 100, random_state = i)

    feat_10.append(forward_selection(model, X_forest, Y_train, cv=k_fold, scoring=lab_accu, feat_only=True))

### Manual feature selection

In [ ]:
sel_feat = manual_selection(feat_10)

Present the final selected features.

In [ ]:
# 1. Minimum E-State
# 2. Topological polar surface area
# 3. Boiling point
# 4. nhigh lowest partial charge weighted BCUTS
# 5. Number of nitrogen atoms
# 6. Number of basic groups
# 7. Overall or summation solute hydrogen bond acidity
# 8. Maximum H E-State

sel_feat = ['gmin', 'TopoPSA', 'BoilingPoint', 'BCUTc-1l', 'nN', 'nBase', 'MLFER_A', 'hmax']

# Transform X
X_sel = X_train[sel_feat]

## Model Selection

Perform model comparison in different feature sets.

In [ ]:
kfold = KFold(n_splits=10, shuffle = True, random_state = 42)

scoring = {'Exact Match Ratio': make_scorer(accuracy_score),
           'Label-based Accuracy': make_scorer(hamming_score)}

In [ ]:
results, names = model_comparison(model_dict, X_train, Y_train, cv=kfold, scoring=scoring)

## Statistical comparison of feature sets

Obtain the accuracy results of the same classifier trained in different feature sets.

The evaluation is done in a 10x10 cross-validation schema.

In [ ]:
# Define classifier and cross-validation schema
dt = DecisionTreeClassifier(random_state=42)
kfold = RepeatedKFold(n_splits=10, n_repeats=10, random_state = 42)

# Scoring (label-based accuracy)
lab_accu = make_scorer(hamming_score)

# Feature sets to test
datasets = [X_train, X_vt, X_corr, X_forest, X_rfe, X_sel]
names = ['Initial', 'Constant Filtering', 'Correlation Filtering', 'RF Feature Importance', 'RFECV', 'Final']

# Perform cross-validation
ds_result = feature_comparison(dt, datasets, Y_train, cv=kfold, scoring=lab_accu, X_names=names)

Apply the Friedman test in the calculated results.

In [ ]:
# Calculate Friedman test
ds_friedman = friedman_test(ds_result)

Since the Friedman test showed statistically significant differences, apply the Nemeyi post hoc test to determine the specific differences. 

In [ ]:
# Calculate Nemeyi test
ds_nemeyi = nemeyi_test(ds_result)

# Show p values for each pair
ds_nemeyi

Finally, compare the initial (1446 features) and final (8 features) sets using Wilcoxon signed rank test.

In [ ]:
# Get the relevant groups
init_perf = ds_result['Initial']
fin_perf = ds_result['Final']

# Calculate Wilcoxon test
wilc_res = wilcoxon_test(fin_perf, init_perf, alternative='one-sided')

## Hyperparameter Optimization

Optimize Decision Tree classifier on final transformed dataset.

In [ ]:
# Transform X
X_sel = X_train[sel_feat]

# Decision Tree Classifier
tree = DecisionTreeClassifier(random_state=42)

# Scoring (label-based accuracy)
lab_accu = make_scorer(hamming_score)

# Cross-validation split
gs_kfold = KFold(n_splits=10, shuffle = True, random_state = 42)

# Parameter grid for Decision Tree Classifier
param_grid = {"criterion": ["gini", "entropy"],
              "max_features": ["auto", "sqrt", "None"],
              "min_samples_split": [2, 3, 5, 8, 10, 20, 40],
              "max_depth": range(3, 30),
              "min_samples_leaf": [1, 2, 3, 4, 5, 10, 20, 40]
              }

In [ ]:
opt_tree = tree_optimization(tree, param_grid, X_sel, Y_train, cv=gs_kfold, scoring=lab_accu)

For label-based accuracy = 0.813, the best estimator is a Decision Tree Classifier with max_depth=26 and max_features='auto'.

Therefore, we need to find the optimal depth for interpretability and performance.

In [ ]:
# Decision Tree Classifier
dtree = DecisionTreeClassifier(max_features='auto', random_state=42)

# Range of depth
depths = list(range(1, 31))

# Scoring (label-based accuracy)
lab_accu = make_scorer(hamming_score)

plot_validation_curve(dtree, X_sel, Y_train, 'max_depth', param_range=depths,
                      cv=gs_kfold, scoring=lab_accu, ylim=(0.7, 1.01), save=True)

plt.show()

## Train classifier and evaluate on test set

### Rule-based classifier - Baseline results

Classification Report

In [ ]:
# Initialize Rule-based Classifier and predict on test set
rule_classifier = RulePredict()
rule_pred = np.vstack(X_test.apply(rule_classifier.predict, axis=1))

title = 'Classification Report - Rule-based Classifier'

# Evaluate
rule_report = general_classification_report(Y_test, rule_pred, title=title)

Confusion Matrix

In [ ]:
cm = multilabel_confusion_matrix(Y_test, rule_pred)

fig, ax = plt.subplots(1, 2, figsize=(12, 7))

fig.suptitle('Confusion Matrix - Rule-based Classifier', y=1.05)

show_confusion_matrix(cm[0], ax[0], 'GC', ['N', 'Y'])
show_confusion_matrix(cm[1], ax[1], 'LC', ['N', 'Y'])

fig.tight_layout()
plt.show()

### Decision Tree results

Classification Report

In [ ]:
# Initialize Decision Tree
dtree = DecisionTreeClassifier(max_depth=4, random_state=42)

# Transform Train/Test set using selected features
sel_feat = ['gmin', 'TopoPSA', 'BoilingPoint', 'BCUTc-1l', 'nN', 'nBase', 'MLFER_A', 'hmax']
X_sel = X_train[sel_feat]
X_test_sel = X_test[sel_feat]

# Train on final dataset
dtree.fit(X_sel, Y_train)

# Predict on test set
tree_pred = dtree.predict(X_test_sel)

# Save model
dump(dtree, 'DecisionTree.joblib')

title = 'Classification Report - Decision Tree Classifier'

# Evaluate
tree_report = general_classification_report(Y_test, tree_pred, title=title)

Confusion Matrix

In [ ]:
cm = multilabel_confusion_matrix(Y_test, tree_pred)

fig, ax = plt.subplots(1, 2, figsize=(12, 7))

fig.suptitle('Confusion Matrix - Decision Tree Classifier', y=1.05)

show_confusion_matrix(cm[0], ax[0], 'GC', ['N', 'Y'])
show_confusion_matrix(cm[1], ax[1], 'LC', ['N', 'Y'])

fig.tight_layout()
plt.show()

### ROC Curve

In [ ]:
classes = ['GC', 'LC']
models = ['Rule-based Classifier', 'Rule-based Classifier', 'Decision Tree Classifier', 'Decision Tree Classifier']

plot_roc_curve(Y_test, rule_pred, tree_pred, models, class_names=classes, save=True)

### Compare classifiers using McNemar's test

Calculate test for GC class.

In [ ]:
# Get GC predictions
gc_true = Y_test[:, 0]
gc_rule_pred = rule_pred[:, 0]
gc_tree_pred = tree_pred[:, 0]

models = ['Rule-based Classifier', 'Decision Tree Classifier']
title = "McNemar's table for GC class"

# Compute test
gc_mcnemar = mcnemar_test(gc_true, gc_rule_pred, gc_tree_pred, model_names=models, title=title)

Calculate test for LC class.

In [ ]:
# Get LC predictions
lc_true = Y_test[:, 1]
lc_rule_pred = rule_pred[:, 1]
lc_tree_pred = tree_pred[:, 1]

models = ['Rule-based Classifier', 'Decision Tree Classifier']
title = "McNemar's table for LC class"

# Compute test
lc_mcnemar = mcnemar_test(lc_true, lc_rule_pred, lc_tree_pred, model_names=models, title=title)

## Visualize tree

In [ ]:
# Export tree to DOT data
dot_data = export_graphviz(dtree, out_file=None,
                           feature_names=X_sel.columns,
                           class_names=['GC', 'LC'],
                           filled=True, proportion=True)

# Draw and save graph
graph = graphviz.Source(dot_data, format='PNG')
graph.render("decision_tree_graph")

graph